In [1]:
import os
import shutil 
import cv2
import json 
import glob
import numpy as np
import random
from tqdm import tqdm 

In [2]:
def calculate_person_scale(bounding_box, reference_height=200):
    """
    Calculate the person scale relative to a reference height.

    Parameters:
    - bounding_box: Tuple or list representing the bounding box coordinates (x, y, width, height).
    - reference_height: Reference height for scaling (default is 200 pixels).

    Returns:
    - Person scale.
    """
    _, _, _, height = bounding_box
    person_scale = height / reference_height
    return person_scale


def augment_image(image, hue_shift, saturation_scale, brightness_scale):
    # Convert the image from BGR to HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Apply augmentation
    hsv_image[:, :, 0] = (hsv_image[:, :, 0] + hue_shift) % 180
    hsv_image[:, :, 1] = np.clip(saturation_scale * hsv_image[:, :, 1], 0, 255)
    hsv_image[:, :, 2] = np.clip(brightness_scale * hsv_image[:, :, 2], 0, 255)

    # Convert the augmented image back to BGR
    augmented_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)

    return augmented_image

# Set augmentation parameters
hue_shifts = [20, -20, 0, 10, 15]  # Degrees to shift the hue
saturation_scales = [1.5, 0.5, 1.0, 1.2, 1.4]  # Scaling factor for saturation
brightness_scales = [1.2, 1.0, 0.8, 1.0, 1.2]  # Scaling factor for brightness

In [8]:
# # File paths
# annotations_file_path = './finalProjectCode/tf-keras-simple-baselines-keypoint-detection/data/mpii/annotations.json'

# # # Read existing annotations from the file
# with open(annotations_file_path, 'r') as file:
#     annotations = json.load(file)
# print( annotations[0])

annotations = []


In [9]:
classes = ['lateralRaises', 'pushups', 'squats']

fol_dir = './customDataset_org/'
output_dir = './customDataset/augImgDataset/'
output_dirCustom = './customDataset/totalCustomDataset/'
validation_split = 0.3
count = 0 
for each_cls in classes:
    list_imgs = glob.glob( fol_dir + each_cls + '/*/*.json')
    print( len(list_imgs))    
    for idx, each in tqdm( enumerate(list_imgs)):        
        # filename         
        with open(each, 'r') as file:
            each_ann = json.load(file)
        image_path = each_ann['imagePath']
        scale = 0 
        # # # get the keypoints 
        cust_ann = {}
        cust_ann['joint_self'] = []
        cust_ann['dataset'] = 'MPI'
        cust_ann['scale_provided'] = scale        
        cust_keypoints = []
        cust_ann['isValidation'] = 0 if random.random() > validation_split else 1
        for each_kp in each_ann['shapes']:            
            if each_kp['shape_type'] == "rectangle":
                [x1, y1, x2, y2] = each_kp['points'][0][0], each_kp['points'][0][1], each_kp['points'][1][0], each_kp['points'][1][1]
                bbox = [x1, y1, y2 - y1, x2 - x1]
                scale = calculate_person_scale( bbox) 
                cust_ann['objpos'] = [(x1 + x2//2), (y1 +  y2)//2]
                continue
            cust_ann['joint_self'].append( [each_kp['points'][0][0], each_kp['points'][0][1], 1])                    
        cust_ann['scale_provided'] = scale
        #print( len( cust_ann['joint_self']), image_path, each_cls)
        diff = 16 - len( cust_ann['joint_self'])
        if diff > 0:
            while len( cust_ann['joint_self']) != 16:
                cust_ann['joint_self'].append( [0, 0, 0])
        if diff < 0:
            while len( cust_ann['joint_self']) != 16:
                cust_ann['joint_self'].pop()        
        assert len( cust_ann['joint_self']) == 16            
        #original_image = cv2.imread(fol_dir + '/' + each_cls + '/' + image_path)
        original_image = cv2.imread(each[:-4] + 'jpg')        
        for i, (hue_shift, saturation_scale, brightness_scale) in enumerate(zip(hue_shifts, saturation_scales, brightness_scales)):            
            augmented_image = augment_image(original_image, hue_shift, saturation_scale, brightness_scale)
            #cv2.imwrite( output_dir + each_cls + '/' + f'augmented_image_{i+1}_{each_cls}_{image_path}.jpg', augmented_image)
            cv2.imwrite( output_dirCustom + f'{count}.jpg', augmented_image)
            each_file_path =  f'augmented_image_{i+1}_{each_cls}_{image_path}'            
            cust_ann['img_paths'] = f'{count}.jpg'
            annotations.append(cust_ann)
            #print( cust_ann['img_paths'], annotations[count], count)                    
            count += 1

50


50it [00:11,  4.31it/s]


50


50it [00:15,  3.13it/s]


47


47it [00:15,  3.07it/s]


In [10]:
# Write the updated annotations back to the file
output_ann_file_path = output_dir + '/annotations.json'
with open(output_ann_file_path, 'w') as file:
    json.dump(annotations, file, indent=2)
print("Dictionary appended and file updated.")

Dictionary appended and file updated.
